# Завдання класифікації тексту

Як ми вже згадували, ми зосередимося на простому завданні класифікації тексту на основі набору даних **AG_NEWS**, яке полягає у класифікації заголовків новин в одну з 4 категорій: Світ, Спорт, Бізнес і Наука/Технології.

## Набір даних

Цей набір даних вбудований у модуль [`torchtext`](https://github.com/pytorch/text), тому ми можемо легко отримати до нього доступ.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Тут `train_dataset` і `test_dataset` містять колекції, які повертають пари мітки (номер класу) і тексту відповідно, наприклад:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Отже, давайте виведемо перші 10 нових заголовків з нашого набору даних:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Оскільки набори даних є ітераторами, якщо ми хочемо використовувати дані кілька разів, нам потрібно перетворити їх на список:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Токенізація

Тепер нам потрібно перетворити текст у **числа**, які можна представити у вигляді тензорів. Якщо ми хочемо отримати представлення на рівні слів, потрібно виконати два кроки:
* використати **токенайзер**, щоб розбити текст на **токени**
* створити **словник** цих токенів.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Використовуючи словник, ми можемо легко закодувати наш токенізований рядок у набір чисел:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Представлення тексту за методом "Мішок слів"

Оскільки слова передають значення, іноді можна зрозуміти зміст тексту, просто аналізуючи окремі слова, незалежно від їхнього порядку в реченні. Наприклад, при класифікації новин слова, такі як *погода*, *сніг*, ймовірно, вказують на *прогноз погоди*, тоді як слова *акції*, *долар* можуть належати до *фінансових новин*.

**Мішок слів** (BoW) — це найпоширеніше традиційне представлення векторів. Кожне слово пов’язане з індексом вектора, а елемент вектора містить кількість появ слова в даному документі.

![Зображення, яке показує, як представлення вектора "Мішок слів" зберігається в пам’яті.](../../../../../translated_images/uk/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Ви також можете уявити BoW як суму всіх векторів з одним активним елементом для окремих слів у тексті.

Нижче наведено приклад того, як створити представлення "Мішок слів" за допомогою бібліотеки Scikit Learn для Python:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Щоб обчислити вектор "мішок слів" з векторного представлення нашого набору даних AG_NEWS, ми можемо використати наступну функцію:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Примітка:** Тут ми використовуємо глобальну змінну `vocab_size` для визначення стандартного розміру словника. Оскільки розмір словника часто досить великий, ми можемо обмежити його до найбільш частотних слів. Спробуйте зменшити значення `vocab_size` і запустити код нижче, щоб побачити, як це впливає на точність. Ви можете очікувати деяке зниження точності, але не драматичне, заради вищої продуктивності.


## Навчання класифікатора BoW

Тепер, коли ми навчилися створювати представлення тексту у вигляді "Мішка слів" (Bag-of-Words), давайте навчимо класифікатор на його основі. Спершу нам потрібно перетворити наш набір даних для навчання таким чином, щоб усі позиційні векторні представлення були перетворені на представлення "Мішка слів". Це можна зробити, передавши функцію `bowify` як параметр `collate_fn` до стандартного `DataLoader` з бібліотеки torch:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Тепер давайте визначимо просту нейронну мережу-класифікатор, яка містить один лінійний шар. Розмір вхідного вектора дорівнює `vocab_size`, а розмір виходу відповідає кількості класів (4). Оскільки ми вирішуємо задачу класифікації, кінцева функція активації — `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Тепер ми визначимо стандартний цикл навчання в PyTorch. Оскільки наш набір даних досить великий, для навчальних цілей ми будемо тренувати лише один епоху, а іноді навіть менше однієї епохи (вказуючи параметр `epoch_size`, ми можемо обмежити навчання). Ми також будемо повідомляти накопичену точність навчання під час тренування; частота звітування визначається за допомогою параметра `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## Біграми, триграми та N-грами

Одним із обмежень підходу "мішок слів" є те, що деякі слова є частиною багатослівних виразів. Наприклад, слово "hot dog" має зовсім інше значення, ніж слова "hot" і "dog" в інших контекстах. Якщо ми завжди представляємо слова "hot" і "dog" однаковими векторами, це може заплутати нашу модель.

Щоб вирішити цю проблему, часто використовуються **представлення N-грам**, особливо в методах класифікації документів, де частота кожного слова, двослівного або трислівного виразу є корисною ознакою для навчання класифікаторів. Наприклад, у представленні біграм ми додаємо всі пари слів до словника, крім оригінальних слів.

Нижче наведено приклад того, як створити представлення "мішок слів" для біграм за допомогою Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Основним недоліком підходу N-грам є те, що розмір словника починає зростати надзвичайно швидко. На практиці нам потрібно поєднувати представлення Н-грам із деякими методами зменшення розмірності, такими як *вбудовування* (embeddings), про які ми поговоримо в наступному розділі.

Щоб використовувати представлення Н-грам у нашому наборі даних **AG News**, нам потрібно створити спеціальний словник Н-грам:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Ми могли б використати той самий код, що й вище, для навчання класифікатора, проте це було б дуже неефективно з точки зору пам’яті. У наступному розділі ми навчимо біграмний класифікатор, використовуючи векторні уявлення.

> **Примітка:** Ви можете залишити лише ті n-грами, які зустрічаються в тексті більше заданої кількості разів. Це забезпечить виключення рідкісних біграм і значно зменшить розмірність. Для цього встановіть параметр `min_freq` на вищу величину і спостерігайте, як змінюється довжина словника.


## Частота терміну та обернена частота документа (TF-IDF)

У представленні BoW (Bag of Words) всі слова мають однакову вагу, незалежно від їх значення. Однак очевидно, що частовживані слова, такі як *a*, *in* тощо, є набагато менш важливими для класифікації, ніж спеціалізовані терміни. Насправді, у більшості завдань обробки природної мови (NLP) деякі слова є більш релевантними, ніж інші.

**TF-IDF** означає **частота терміну – обернена частота документа**. Це варіація моделі "мішок слів", де замість бінарного значення 0/1, яке вказує на наявність слова в документі, використовується число з плаваючою точкою, що пов'язане з частотою появи слова в корпусі.

Більш формально, вага $w_{ij}$ слова $i$ в документі $j$ визначається як:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
де
* $tf_{ij}$ — кількість появ слова $i$ в документі $j$, тобто значення BoW, яке ми бачили раніше
* $N$ — кількість документів у колекції
* $df_i$ — кількість документів, що містять слово $i$ у всій колекції

Значення TF-IDF $w_{ij}$ зростає пропорційно до кількості разів, коли слово з'являється в документі, і зменшується залежно від кількості документів у корпусі, які містять це слово. Це допомагає врахувати той факт, що деякі слова зустрічаються частіше за інші. Наприклад, якщо слово з'являється *у кожному* документі колекції, то $df_i=N$, і $w_{ij}=0$, і такі терміни будуть повністю ігноруватися.

Ви можете легко створити векторизацію TF-IDF тексту за допомогою Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Висновок

Хоча представлення TF-IDF надають вагу частоті різних слів, вони не здатні передати значення або порядок. Як сказав відомий лінгвіст Дж. Р. Фьорт у 1935 році: "Повне значення слова завжди є контекстуальним, і жодне вивчення значення поза контекстом не може бути сприйняте серйозно." Пізніше в курсі ми дізнаємося, як захоплювати контекстуальну інформацію з тексту за допомогою мовного моделювання.



---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають у результаті використання цього перекладу.
